# Práctica 1: Web Scraping
(variante con envío a base de datos)

Conexión a la web de conteo de Coronavirus detallada por país:
https://www.worldometers.info/coronavirus/

Se utilizará Beautiful Soup para hacer Web Scraping, buscando la tabla de datos.
Pandas para pasar la información a un dataframe y ajustar la información para poder enviarla a una base de datos.
Con esta técnica se puede hacer una solución en línea que haga análisis y visualización de los datos casi en tiempo real.
Si se convierte en un script, se puede ejecutar cada minuto y quedar así sincronizado con la fuente de datos.

## Sección 1: Web Scrapping

En esta sección del código se busca la tabla del origen y se guarda en una tabla.

In [1]:
#importamos las librerías de BeautifulSoup para obtener el código de html y ayudarnos al scraping
#se imprta de librería de request para "conectarse" a la url

from bs4 import BeautifulSoup as soup
from requests import get

#el dataset se encuentra en este página, la cual es actualizada cada minuto
my_url = "https://www.worldometers.info/coronavirus/"

pageHTML = get(my_url)

In [2]:
#se obtiene el código html
soup = soup(pageHTML.text)

#se muestran los primeros 1000 caracteres
soup.text[:1000]

'\n\n\n\n\n\nCoronavirus Update (Live): 1,888,904 Cases and 117,585 Deaths from COVID-19 Virus Pandemic - Worldometer\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t$.extend( $.fn.dataTable.defaults, {\n    responsive: true\n} );\n\t\n\t$(document).ready(function() {\n    $(\'#example2\').dataTable( {\n        "scrollCollapse": true,\n\t\t"sDom": \'<"bottom"flp><"clear">\',\n        "paging":         false\n    } );\n} );\n\t\n\n\t\t$.extend( $.fn.dataTable.defaults, {\n    responsive: true\n} );\n\t\n\t$(document).ready(function() {\n    $(\'#table3\').dataTable( {\n        "scrollCollapse": true,\n\t\t\t\t "order": [[ 1, \'desc\' ]],\n\t\t"sDom": \'<"bottom"flp><"clear">\',\n        "paging":         false\n    } );\n} );\n\t\n\n\t\t$.extend( $.fn.dataTable.defaults, {\n    responsive: true\n} );\n\t\n\t$(document).ready(function() {\n    $(\'#example\').dataTable( {\n        "scrollCollapse": true,\n\t\t"searching": false,\n\n\t\t"sDom": \'<"top">rt<"bottom"flp><

In [3]:
# con este código obtenemos la tabla que contiene la información de detalle de país que es la que queremos obtener.
# previo a escribir este código, se hizo una búsqueda manual en la página mirando el código de html para ver
# en dónde se encontraba la información, esta revisión se hizo con el browser, utilizando las herramientas de desarrollador
# que permiten ubicar el código señalando el objeto de interés, en este caso la tabla. Para esto utilicé Chrome con las
# herramientas de desarrollador y encontré que el objeto 'table' con la etiqueta 'tbody' contenía los valores
data = []
table = soup.find('table')
table_body = table.find('tbody')

# se van ubicando los saltos de línea para poder ir uno a uno llenando las líneas de la variable de tabla en donde 
# se guardarán los datos
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

# se muestran los valores de la tabla para ver cómo viene la información.    
print (data)

[['North America', '605,821', '+7,526', '24,296', '+820', '43,697', '537,828', '12,897', '', '', '', '', 'North America'], ['Europe', '899,365', '+18,255', '78,929', '+2,161', '230,405', '590,031', '29,843', '', '', '', '', 'Europe'], ['Asia', '308,207', '+9,471', '11,301', '+299', '150,330', '146,576', '6,651', '', '', '', '', 'Asia'], ['South America', '51,561', '+980', '2,162', '+81', '6,085', '43,314', '1,142', '', '', '', '', 'South America'], ['Oceania', '7,797', '+65', '66', '+3', '4,041', '3,690', '84', '', '', '', '', 'Australia/Oceania'], ['Africa', '15,432', '+350', '818', '+27', '2,999', '11,615', '178', '', '', '', '', 'Africa'], ['', '721', '', '13', '', '619', '89', '10', '', '', '', '', ''], ['World', '1,888,904', '+36,647', '117,585', '+3,391', '438,176', '1,333,143', '50,805', '242', '15.1', '', '', 'All'], ['USA', '566,654', '+6,354', '22,877', '+772', '33,743', '510,034', '11,807', '1,712', '69', '2,872,678', '8,679', 'North America'], ['Spain', '169,496', '+2,665',

In [4]:
# eliminamos los valores que no tienen datos, si es que hay alguno
missingData = []
fixedData = [[]]
for row in data:
    for index,col in enumerate(row):
        if(col == ''):
            row[index] = 0
        
    missingData.append(row)

In [5]:
# definimos las cabeceras de acuerdo a la tabla de la web, colocando el nombre que queramos para las columnas.
# debemos tener cuidado porque la página fuente es actualizada constantemente, desde que inicié la práctica, se han hecho
# 5 cambios y se añaden líneas o columnas y si las columnas no coinciden, esta parte del código debe ser revisada y ajustada
# ya que de otra forma enviará un error, la forma de evitar el error es revisar el nombre de las columnas en la página fuente
# y dejar igual las columnas en un sitio y otro, si hay columnas o líneas no útiles, más adeoante se pueden eliminar.

headers = ["Country", "Total Cases", "New Cases", "Total Deaths","New Deaths", "Total Recovered", "Active Cases", "Serious, Critical", "Tot Cases/1M pop", "Tot Deaths/1M pop", "Total Tests", "Tests/1M pop", "Region"]

# mostramos las cabeceras para ver que todo es correcto
headers

# hasta este punto se ha hecho el scraping y la información está lista para pasar a un dataframe

['Country',
 'Total Cases',
 'New Cases',
 'Total Deaths',
 'New Deaths',
 'Total Recovered',
 'Active Cases',
 'Serious, Critical',
 'Tot Cases/1M pop',
 'Tot Deaths/1M pop',
 'Total Tests',
 'Tests/1M pop',
 'Region']

## Sección 2: Envío de datos a data frame

En esta sección se hace limpieza de datos y se escribe la tabla extraida a un dataframe que permite hacer tratamiento y operaciones sobre los datos.

In [6]:
# importamos pandas y numpy para crear dataframes a partir de la tabla creada y poder manipular mejor la información
import pandas as pd
import numpy as np

# se envía la tabla al dataframe
df = pd.DataFrame()
df = df.append(data)
df.columns = headers

# se muestra la información en formato dataframe
df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/1M pop,Tot Deaths/1M pop,Total Tests,Tests/1M pop,Region
0,North America,"605,821","+7,526","24,296",+820,"43,697","537,828","12,897",0,0,0,0,North America
1,Europe,"899,365","+18,255","78,929","+2,161","230,405","590,031","29,843",0,0,0,0,Europe
2,Asia,"308,207","+9,471","11,301",+299,"150,330","146,576","6,651",0,0,0,0,Asia
3,South America,"51,561",+980,"2,162",+81,"6,085","43,314","1,142",0,0,0,0,South America
4,Oceania,"7,797",+65,66,+3,"4,041","3,690",84,0,0,0,0,Australia/Oceania
5,Africa,"15,432",+350,818,+27,"2,999","11,615",178,0,0,0,0,Africa
6,0,721,0,13,0,619,89,10,0,0,0,0,0
7,World,"1,888,904","+36,647","117,585","+3,391","438,176","1,333,143","50,805",242,15.1,0,0,All
8,USA,"566,654","+6,354","22,877",+772,"33,743","510,034","11,807","1,712",69,"2,872,678","8,679",North America
9,Spain,"169,496","+2,665","17,489",+280,"64,727","87,280","7,371","3,625",374,"600,000","12,833",Europe


In [7]:
## Elimina los signos que impiden convertir a número

df['Total Cases'] = df['Total Cases'].str.replace(',', '')
df['New Cases'] = df['New Cases'].str.replace('+', '')
df['New Cases'] = df['New Cases'].str.replace(',', '')
df['Total Deaths'] = df['Total Deaths'].str.replace(',', '')
df['New Deaths'] = df['New Deaths'].str.replace('+', '')
df['New Deaths'] = df['New Deaths'].str.replace(',', '')
df['Total Recovered'] = df['Total Recovered'].str.replace(',', '')
df['Active Cases'] = df['Active Cases'].str.replace(',', '')
df['Serious, Critical'] = df['Serious, Critical'].str.replace(',', '')
df['Tot Cases/1M pop'] = df['Tot Cases/1M pop'].str.replace(',', '')
df['Tot Deaths/1M pop'] = df['Tot Deaths/1M pop'].str.replace(',', '')

#caso irregular ocurrido el 13 de abril:
df['Total Recovered'] = df['Total Recovered'].str.replace('N/A', '0')

## Elimina los NaN para valores nulos y los sustituye por ceros
df.fillna(0, inplace=True)

In [8]:
# muestra los tipos de dato que de momento todo son object
df.dtypes

Country              object
Total Cases          object
New Cases            object
Total Deaths         object
New Deaths           object
Total Recovered      object
Active Cases         object
Serious, Critical    object
Tot Cases/1M pop     object
Tot Deaths/1M pop    object
Total Tests          object
Tests/1M pop         object
Region               object
dtype: object

In [9]:
# cambia los tipos de dato del dataframe, para poder enviar el dataframe eventualmente a una base de datos.
# en caso de enviarlo a un csv, este paso no tiene tanto impacto, sin embargo si queremos hacer operaciones con el
# dataframe, lo mejor es tener las columas declaradas en el formato correcto:

df = df.astype({"Total Cases": int})
df = df.astype({"New Cases": int})
df = df.astype({"Total Deaths": int})
df = df.astype({"New Deaths": int})
df = df.astype({"Total Recovered": int})
df = df.astype({"Active Cases": int})
df = df.astype({"Serious, Critical": int})
df = df.astype({"Tot Cases/1M pop": float})
df = df.astype({"Tot Deaths/1M pop": float})

# vemos que se han convertido correctamente
df.dtypes

Country               object
Total Cases            int32
New Cases              int32
Total Deaths           int32
New Deaths             int32
Total Recovered        int32
Active Cases           int32
Serious, Critical      int32
Tot Cases/1M pop     float64
Tot Deaths/1M pop    float64
Total Tests           object
Tests/1M pop          object
Region                object
dtype: object

In [10]:
# Removemos columnas y líneas no utilizadas
# es importante señalar que para este caso, he decidido quitar estas columnas, porque el scraping del covid está
# teniendo mucho movimiento porque al tratarse de un tema de interés mundial, la página fuente altera la tabla constantemente
# para aportar más información y en nuestra párctica no toda la información es relavante

# elimino 3 columnas que no estaban en el código inicial.
# en casos futuros, si 

# quita las columnas no utilizadas.
del df['Total Tests']
del df['Tests/1M pop']
del df['Region']

# y quita las líneas no utilizadas que corresponden a agrupaciones y sumas que se pueden hacer por otros medios.
# si las dejamos enviará valores duplicados al hacer sumas.

i = 0
while i < 8:
  df.drop(df.index[0],inplace=True)
  i += 1

# reinicia el índice del dataframe     
df.reset_index(drop=True, inplace=True)    
    
# mostramos y validamos la tabla en formato final
df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,"Serious, Critical",Tot Cases/1M pop,Tot Deaths/1M pop
0,USA,566654,6354,22877,772,33743,510034,11807,1712.00,69.00
1,Spain,169496,2665,17489,280,64727,87280,7371,3625.00,374.00
2,Italy,159516,3153,20465,566,35435,103616,3260,2638.00,338.00
3,France,132591,0,14393,0,27186,91012,6845,2031.00,221.00
4,Germany,127916,62,3022,0,64300,60594,4895,1527.00,36.00
5,UK,88621,4342,11329,717,0,76948,1559,1305.00,167.00
6,Iran,73303,1617,4585,111,45983,22735,3877,873.00,55.00
7,Turkey,61049,4093,1296,98,3957,55796,1786,724.00,15.00
8,Belgium,30589,942,3903,303,6707,19979,1234,2639.00,337.00
9,Netherlands,26551,964,2823,86,250,23478,1384,1550.00,165.00


## Sección 3: Comprobaciones y envío a base de datos

En esta sección se comprueba que los valores son correctos vs la fuente y se envían los datos a una base de datos para su posterior uso, en otras herramientas de análisis, visualización o predicción.

In [11]:
## Pone el resumen de datos para validar que efectivamente es correcto con respecto a la página fuente

Total_Cases= df['Total Cases'].sum()
Total_Deaths= df['Total Deaths'].sum()
Total_Recovered = df['Total Recovered'].sum()


print ("Total Cases=",Total_Cases)
print ("Total Deaths=",Total_Deaths)
print ("Total Recovered=",Total_Recovered)

Total Cases= 1888904
Total Deaths= 117585
Total Recovered= 437832


In [58]:
# importa librerías para conexión a base de datos (en este ejemplo es postgresql) y también hay liberías para
# ejecución de código sql y envío de dataframe a una tabla.

from sqlalchemy import create_engine
import psycopg2 
import io

# parámetros de conexión, se pueden poner en un fichero de configuración se deben de sustituir los parámetros entre <>
# por los que correspondan
engine = create_engine('postgresql+psycopg2://<usuario>:<password>@<host>:<puerto>/<base_de_datos>')

# en este caso la tabla se reescribe cada vez que el script sea ejecutado, para evitar duplicidad
df.to_sql('<tabla>', engine, if_exists='replace', index=False)

conn = engine.raw_connection()
cur = conn.cursor()
conn.commit()